In [1]:
# pip install sentence_transformers==2.7.0

In [2]:
from sentence_transformers import SentenceTransformer

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

## Q1. Getting the embeddings model
### What's the first value of the resulting vector?

In [4]:
user_question = "I just discovered the course. Can I still join it?"

In [6]:
v= embedding_model.encode(user_question)

In [7]:
len(v)

768

In [8]:
v[0]

0.078222655

## Prepare the documents

In [10]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [17]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [16]:
len(documents)

948

In [14]:
# Desired course to filter
desired_course = 'machine-learning-zoomcamp'

# Filter documents
filtered_documents = [doc for doc in documents if doc['course'] == desired_course]

In [15]:
len(filtered_documents)

375

## Q2. Creating the embeddings
###  What's the shape of X? (X.shape). Include the parantheses.

In [18]:
import numpy as np
from tqdm.auto import tqdm

In [20]:
embeddings = []

for doc in tqdm(filtered_documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [01:04<00:00,  5.84it/s]


In [21]:
X.shape

(375, 768)

## Q3. Search
### What's the highest score in the results?

In [22]:
scores = X.dot(v)

In [25]:
max_score = np.max(scores)

In [26]:
max_score

0.6506573

### Vector search
### We can now compute the similarity between a query vector and all the embeddings.

### Let's use this to implement our own vector search

In [27]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine
### Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.
### First, load the ground truth dataset:

In [28]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [29]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [30]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [32]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        v = embedding_model.encode(q['question'])
        results = search_function(v, num_results=5)   
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        # 'mrr': mrr(relevance_total),
    }

### Use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5. What did you get?

In [33]:
evaluate(ground_truth, search_engine.search)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:27<00:00, 20.83it/s]


{'hit_rate': 0.9398907103825137}